In [1]:
from pathlib import Path
from tqdm import tqdm
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb

import rebound as rb
import sys
sys.path.append('../')
from linear_theory import linear_theory_prediction, make_simpler_secular_theory

plt.style.use('../paper.mplstyle')

base_path = Path("../data")

In [2]:
uncertainty_asteroids_sampled = pd.read_csv(base_path/"uncertainty_asteroids_sampled.csv")
# merge asteroid index and designator into one column
uncertainty_asteroids_sampled["Des'n"] = uncertainty_asteroids_sampled["Des'n"] + "-" + uncertainty_asteroids_sampled[uncertainty_asteroids_sampled.columns[0]].astype(str)
uncertainty_asteroids_sampled = uncertainty_asteroids_sampled.drop(columns=[uncertainty_asteroids_sampled.columns[0]])

uncertainty_asteroids_proper_elements = pd.read_csv(base_path/"uncertainty_asteroid_elements_proper.csv", index_col=0)

uncertainty_asteroids = pd.merge(uncertainty_asteroids_sampled, uncertainty_asteroids_proper_elements, on="Des'n", how="inner")
uncertainty_asteroids.head()

,Des'n,epoch,x,y,z,vx,vy,vz,g,s,propa,prope,propsini
0,K23Q65H-0,2460193.5,2.089677,-0.471412,-0.050662,1.119444,4.371910,-0.160598,39.448867,-40.817830,2.399421,0.076069,0.033174
1,K23Q65H-1,2460193.5,2.083370,-0.470422,-0.050383,1.126286,4.379951,-0.159831,39.281278,-40.698449,2.395599,0.077768,0.032973
2,K23Q65H-2,2460193.5,2.104747,-0.474021,-0.051316,1.113403,4.351187,-0.162884,39.664863,-41.193396,2.409175,0.074555,0.033692
3,K23Q65H-3,2460193.5,2.103007,-0.473237,-0.051251,1.105546,4.355013,-0.162204,39.584892,-41.067235,2.406828,0.073510,0.033651
4,K23Q65H-4,2460193.5,2.084378,-0.470628,-0.050423,1.114949,4.380133,-0.159520,39.215266,-40.612570,2.394750,0.075792,0.033164


In [3]:
final_model_e = xgb.XGBRegressor()
final_model_e.load_model(base_path/"models/best_model_e_final.xgb")
final_model_inc = xgb.XGBRegressor()
final_model_inc.load_model(base_path/"models/best_model_inc_final.xgb")

simpler_secular_theory = make_simpler_secular_theory()

In [4]:
g0 = []
s0 = []
prope_h = []
propsini_h = []
for row,sample in tqdm(uncertainty_asteroids.iterrows(), total=uncertainty_asteroids.size):
    # load osculating elements to feed into xgboost
    sim = rb.Simulationarchive(str(base_path/"uncertainty_integrations"/f"asteroid_integration_{sample["Des'n"]}.sa"))[0]

    u, v, g, s = linear_theory_prediction(sim.particles[-1].e, sim.particles[-1].inc, sim.particles[-1].omega, sim.particles[-1].Omega, sim.particles[-1].a,
                             sample['propa'], simpler_secular_theory)

    g0.append(g)
    s0.append(s)
    prope_h.append(np.abs(u))
    prope_h.append(np.abs(v))

  0%|          | 6/35516 [00:13<22:22:37,  2.27s/it]


KeyboardInterrupt: 